<h1 style="background: linear-gradient(180deg,rgb(92, 0, 128) 0%,rgb(46, 0, 153) 75%, rgb(65, 0, 230) 100%); color: white; font-family: 'Raleway', sans-serif; padding: 10px 20px; border-radius: 10px; text-align: center; font-weight:500;">
Representaciones de Texto
</h1>
<br>

**PRESENTAN** Armando Islas, Daniela Flores, Oscar Berrueco, Natalia Anaya

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import logging
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from pathlib import Path

In [4]:
# Directorios de entrada y salida
INPUT_DIR = "../outputs/normalized/"
OUTPUT_DIR = "../outputs/text-reps/"

In [6]:
# Crear directorio de salida si no existe
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [7]:
# Configuración de parámetros
NGRAM_RANGES = [(1, 1), (2, 2), (3, 3)]
NGRAM_NAMES = ['uni', 'bi', 'tri']
VECTORIZER_MODES = ['binary', 'freq', 'tfidf']
SVD_COMPONENTS = 50
RANDOM_STATE = 0

In [9]:
def get_vectorizer(mode, ngram_range):
    """
    Devuelve el vectorizador apropiado según el modo y el rango de n-gramas
    
    Args:
        mode (str): Modo de vectorización ('binary', 'freq', 'tfidf')
        ngram_range (tuple): Rango de n-gramas como tupla (n, n)
        
    Returns:
        Vectorizador configurado
    """
    if mode == 'binary':
        return CountVectorizer(ngram_range=ngram_range, binary=True)
    elif mode == 'freq':
        return CountVectorizer(ngram_range=ngram_range)
    elif mode == 'tfidf':
        return TfidfVectorizer(ngram_range=ngram_range)
    else:
        raise ValueError(f"Modo de vectorización no válido: {mode}")

In [10]:
def process_file(file_path):
    """
    Procesa un archivo, generando todas las representaciones vectoriales y aplicando SVD
    
    Args:
        file_path (str): Ruta al archivo CSV de entrada
    """
    try:
        # Obtener nombre base del archivo
        base_name = os.path.basename(file_path).split('.')[0]
        
        # Leer datos
        df = pd.read_csv(file_path)
        
        # Verificar columnas
        if 'Teaser Text' not in df.columns or 'Tag Value' not in df.columns:
            return
        
        # Separar características y etiquetas
        X_text = df['Teaser Text'].fillna('').values
        y = df['Tag Value'].values
        
        # Iterar sobre configuraciones de n-gramas y modos de vectorización
        for ngram_idx, ngram_range in enumerate(NGRAM_RANGES):
            ngram_name = NGRAM_NAMES[ngram_idx]
            
            for mode in VECTORIZER_MODES:
                
                # Obtener vectorizador
                vectorizer = get_vectorizer(mode, ngram_range)
                
                # Generar matriz de características
                X_vectorized = vectorizer.fit_transform(X_text)
                
                # Aplicar TruncatedSVD para reducción de dimensionalidad
                svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=RANDOM_STATE)
                X_svd = svd.fit_transform(X_vectorized)
                
                # Calcular varianza explicada
                explained_variance = svd.explained_variance_ratio_.sum()
                
                # Crear el objeto de resultado para guardar
                result = {
                    'X': X_svd,
                    'y': y,
                    'vectorizer': vectorizer,
                    'svd': svd,
                    'explained_variance': explained_variance,
                    'config': {
                        'ngram_range': ngram_range,
                        'mode': mode,
                        'svd_components': SVD_COMPONENTS
                    }
                }
                
                # Definir nombre de archivo de salida
                output_filename = f"{base_name}_{ngram_name}_{mode}_svd{SVD_COMPONENTS}.pkl"
                output_path = os.path.join(OUTPUT_DIR, output_filename)
                
                # Guardar resultado
                with open(output_path, 'wb') as f:
                    pickle.dump(result, f)
    
    except Exception as e:
        raise e

In [11]:
# Verificar que exista el directorio de entrada
if not os.path.exists(INPUT_DIR):
    print('No existe el directorio')

# Enumerar archivos CSV en el directorio de entrada
csv_files = [os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) 
             if f.endswith('.csv') and f.startswith('normalized_')]

if not csv_files:
    print('No hay archivos CSV en el directorio')

# Procesar cada archivo
for file_path in csv_files:
    process_file(file_path)